# Expert Model

In [ ]:
%%bash
git clone https://github.com/sunnytqin/no-distillation.git

In [ ]:
%%bash
mkdir -p /kaggle/working/expert_results

In [ ]:
%%bash
mkdir -p /kaggle/working/data/tiny-imagenet-200

SRC=tiny-imagenet-200

cp -r /kaggle/input/tiny-imagenet-200/$SRC/* /kaggle/working/data/tiny-imagenet-200/

In [ ]:
%%bash
cd /kaggle/working/data/tiny-imagenet-200/val

awk '{print $1, $2}' val_annotations.txt | \
while read IMG CLS; do
  mkdir -p images/$CLS
  mv images/$IMG images/$CLS/
done

In [ ]:
%%bash
export PYTHONPATH=/kaggle/working/no-distillation:/kaggle/working/no-distillation/softlabel:$PYTHONPATH

python /kaggle/working/no-distillation/train_expert/buffer.py \
  --dataset Tiny \
  --model ConvNetD4 \
  --train_epochs 90 \
  --num_experts 1 \
  --buffer_path /kaggle/working/no-distillation/results_100_S/checkpoint_epoch_11.pth \
  --data_path /kaggle/working/data/tiny-imagenet-200 \
  --save_interval 1

In [ ]:
print("\n\n === DONE TRAINING EXPERT === \n\n")

# Data-Knowledge Scaling Law (Expert 11)

In [ ]:
%%bash
cd /kaggle/working/no-distillation

cp -r softlabel/indices .


ls -R indices


In [ ]:
%%bash
cd /kaggle/working/no-distillation

mkdir -p results_100_S/Tiny/imagenette/ConvNetD4

cp \
  results_100_S/checkpoint_epoch_11.pth/Tiny/ConvNetD4/replay_buffer_0.pt \
  results_100_S/Tiny/imagenette/ConvNetD4/replay_buffer_1.pt

cp \
  results_100_S/checkpoint_epoch_11.pth/Tiny/ConvNetD4/replay_buffer_0.pt \
  results_100_S/checkpoint_epoch_11.pth/Tiny/ConvNetD4/replay_buffer_1.pt

In [ ]:
%%bash
cd /kaggle/working/no-distillation
export PYTHONPATH=$(pwd):$(pwd)/train_expert:$PYTHONPATH

python softlabel/nodistill.py \
  --dataset Tiny \
  --ipc 1 \
  --expt_type data_knowledge_scaling \
  --teacher_label \
  --max_expert_epoch 11 \
  --lr_net 1e-2 \
  --expert_path /kaggle/working/no-distillation/results_100_S \
  --data_path /kaggle/working/data/tiny-imagenet-200 \
  --student_model ConvNetD4 \
  --teacher_model ConvNetD4 \
  --epoch_eval_train 3000 \
  --optimizer=SGD